In [0]:
import pyspark
from pyspark.sql.functions import *

In [0]:
spark.conf.set(
    "fs.azure.account.key.nnrgsblobacc.dfs.core.windows.net",
    "KOC7m5T3SJnlbUb4P19ep2pbo2PIYAdadSUN9EOxSbcvp7y5OPOAR7ozRXFqnC2hJrh8WA1B+V3E+AStoZvUNQ=="
)

In [0]:
#display(dbutils.fs.ls("abfss://assesment@nnrgsblobacc.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/fact.averagecosts.dlm,fact.averagecosts.dlm,29142959,1713418287000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/fact.transactions.dlm,fact.transactions.dlm,380631122,1713418336000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.clnd.dlm,hier.clnd.dlm,276119,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.hldy.dlm,hier.hldy.dlm,545,1713418278000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.invloc.dlm,hier.invloc.dlm,2702,1713418278000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.invstatus.dlm,hier.invstatus.dlm,17004,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.possite.dlm,hier.possite.dlm,5079,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.pricestate.dlm,hier.pricestate.dlm,157,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.prod.dlm,hier.prod.dlm,194621,1713418280000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.rtlloc.dlm,hier.rtlloc.dlm,3456,1713418280000


In [0]:
data = spark.read.option("header", True).option('inferSchema', True).option("delimiter", "|").csv('abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.hldy.dlm')


In [0]:
#len(data.columns), data.count()

(2, 19)

In [0]:
#data.display()

hldy_id,hldy_label
Valentines_Day,Valentine's Day
Presidents_Day,Presidents Day
St_Patricks_Day,St. Patrick's Day
Easter,Easter
Mothers_Day,Mother's Day
Memorial_Day,Memorial Day
Fathers_Day,Father's Day
Independence_Day,Independence Day
Labor_Day,Labor Day
Rosh_Hashanah,Rosh Hashanah


In [0]:
#data.dtypes

[('hldy_id', 'string'), ('hldy_label', 'string')]

In [0]:
data.drop_duplicates()

DataFrame[hldy_id: string, hldy_label: string]

In [0]:
# Boiler plate code for specific operation
# write cleaned data to the staging dat
output_container_path = 'abfss://stagingdata@nnrgsblobacc.dfs.core.windows.net/'
data.coalesce(1).write.mode('append').option('header' ,'True').format('com.databricks.spark.csv').save(output_container_path)
files = dbutils.fs.ls(output_container_path) 
output_file = [x for x in files if x.name.startswith('part-')]  # To read only part files
dbutils.fs.mv(output_file[0].path,"%s/DimHldy.csv" % output_container_path )
files_to_delete = [file.path for file in files if file.name.startswith('_')]
for file_path in files_to_delete:
    dbutils.fs.rm(file_path, True)